In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pytz
from datetime import datetime, timedelta
import yfinance as yf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Function to load data and model based on user selection
def load_data_and_model(n_days):
    # Load the appropriate model based on the number of days selected
    if n_days == 1:
        model = load_model('C:\\Users\\Sarrang\\Desktop\\lstm_btc_V2.h5')
        # Fetch hourly data for the last 60 hours
        end_date = datetime.now(pytz.timezone('Asia/Kolkata'))
        start_date = end_date - timedelta(hours=60)
        interval = '1h'
    else:
        model = load_model('lstm_btc_V1.h5')
        # Fetch daily data for the last 60 days
        end_date = datetime.now(pytz.timezone('Asia/Kolkata'))
        start_date = end_date - timedelta(days=60)
        interval = '1d'

    # Download Bitcoin data
    data = yf.download('BTC-USD', start=start_date, end=end_date, interval=interval)
    if data.index.tzinfo is None:  # If index is naive
        data.index = data.index.tz_localize(pytz.utc).tz_convert('Asia/Kolkata')
    else:
        data.index = data.index.tz_convert('Asia/Kolkata')  # Convert to IST if already timezone-aware

    # Scale the data
    data = data[['Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    # Create sequences for prediction
    sequence_length = min(len(scaled_data) - 1, 60)
    X = np.array([scaled_data[i:i + sequence_length] for i in range(len(scaled_data) - sequence_length)])
    if X.ndim == 2:
        X = X.reshape((X.shape[0], X.shape[1], 1))
    
    last_data = scaled_data[-sequence_length:].reshape((1, sequence_length, 1))

    return model, last_data, scaler, data.index[-1]


In [14]:
# Function to forecast future prices
def forecast_future_prices(model, last_data, n_periods, scaler, last_date, period='hours'):
    predictions = []
    dates = []
    current_date = last_date
    
    for _ in range(n_periods):
        pred_scaled = model.predict(last_data)
        pred = scaler.inverse_transform(pred_scaled).flatten()
        predictions.append(pred[0])
        
        if period == 'hours':
            current_date = current_date + pd.Timedelta(hours=1)
        else:
            current_date = current_date + pd.Timedelta(days=1)
        
        dates.append(current_date)
        last_data = np.roll(last_data, shift=-1, axis=1)
        last_data[0, -1, 0] = pred_scaled[0, 0]

    return dates, predictions

In [21]:
def on_button_click(b):
    clear_output(wait=True)
    n_days = int(dropdown.value)
    
    model, last_data, scaler, last_date = load_data_and_model(n_days)
    n_periods = n_days if n_days > 1 else n_days * 24  # Hours for 1D, days for others
    period = 'days' if n_days > 1 else 'hours'
    
    dates, predictions = forecast_future_prices(model, last_data, n_periods, scaler, last_date, period)

    # Create a DataFrame for the forecast
    forecast_df = pd.DataFrame({'Datetime': dates, 'Predicted Price': predictions})
    
    # Create an interactive plot with Plotly
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=forecast_df['Datetime'], y=forecast_df['Predicted Price'],
                             mode='lines+markers', marker=dict(color='blue'), line=dict(width=2)))
    
    if n_days == 1:
        fig.update_layout(title='Bitcoin Price Forecast for Next 24 Hours', 
                          xaxis_title='Time of Day', yaxis_title='Predicted Price', 
                          xaxis_tickformat='%H:%M')
    else:
        fig.update_layout(title=f'Bitcoin Price Forecast for Next {n_days} Days', 
                          xaxis_title='Date', yaxis_title='Predicted Price')
        
        fig.update_xaxes(
            tickformat='%Y-%m-%d',
            dtick='D1' if n_days <= 5 else ('D2' if n_days <= 15 else ('D5' if n_days <= 30 else 'M1')),  # Adjust tick interval
            tickangle=-45,
            tickvals=forecast_df['Datetime'][::max(1, len(forecast_df) // 20)],  # Show fewer ticks if too many
            ticktext=[date.strftime('%Y-%m-%d') for date in forecast_df['Datetime'][::max(1, len(forecast_df) // 20)]]
        )

    fig.update_xaxes(rangeslider_visible=True)
    fig.show()


In [25]:
# Dropdown for user to select the number of days to forecast
dropdown = widgets.Dropdown(
    options=[('1 Day', 1), ('5 Days', 5), ('15 Days', 15), ('20 Days', 20), ('30 Days', 30)],
    value=1,
    description='Days to Forecast:',
    style={'description_width': 'initial'}
)
display(dropdown)

# Button to execute the forecast
button = widgets.Button(description="Forecast", button_style='info')
button.on_click(on_button_click)
display(button)

Dropdown(description='Days to Forecast:', options=(('1 Day', 1), ('5 Days', 5), ('15 Days', 15), ('20 Days', 2…

Button(button_style='info', description='Forecast', style=ButtonStyle())